In [ ]:
# Cell 1: Imports and Mock Setup
import pandas as pd
from datetime import datetime
from unittest.mock import MagicMock

from process_capture import process_capture
from fusion_core import fuse_alerts, display_outputs, trigger_alert, DEFAULT_WEIGHTS

# Mock database collections
ts_collection = MagicMock()
geo_collection = MagicMock()
users_collection = MagicMock()
ts_collection.find.return_value = [{"hour": 12},  {"lat": 40.7128}, {"lon": -74.0060}]

"""# Mock model functions
def capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):
    return datetime.now()

def load_threshold_model(user_id):
    return None

def predict_threshold(model, features):
    return 0.05

def detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):
    return 0.0, 0.6  # loc_anomaly, time_anomaly

def load_incident_model(user_id):
    return None, None

def predict_incident(model, scaler, loc_anomaly, time_anomaly):
    return 0.5

def predict_crime_risk(lat, lon):
    return 0.7

def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.8

def predict_audio_stress(user_id):
    return 0.4

def detect_emergency_keyword(user_id):
    return 0.0

def insert_user_alert(users_collection, user_id, loc_anomaly, time_anomaly, incident_prob, trigger_alert):
    print(f"Stored alert for {user_id}")"""
#INSTEAD HERE WE NEED TO CALL OUR MODELS.


'# Mock model functions\ndef capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):\n    return datetime.now()\n\ndef load_threshold_model(user_id):\n    return None\n\ndef predict_threshold(model, features):\n    return 0.05\n\ndef detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):\n    return 0.0, 0.6  # loc_anomaly, time_anomaly\n\ndef load_incident_model(user_id):\n    return None, None\n\ndef predict_incident(model, scaler, loc_anomaly, time_anomaly):\n    return 0.5\n\ndef predict_crime_risk(lat, lon):\n    return 0.7\n\ndef detect_movement_anomaly(user_id, ts_collection, lat, lon, now):\n    return 0.8\n\ndef predict_audio_stress(user_id):\n    return 0.4\n\ndef detect_emergency_keyword(user_id):\n    return 0.0\n\ndef insert_user_alert(users_collection, user_id, loc_anomaly, time_anomaly, incident_prob, trigger_alert):\n    print(f"Stored alert for {user_id}")'

IMPORTING THE FIRSTMODEL: HIGH RISK LOCATOR

In [ ]:
import joblib
from pathlib import Path


#DBSCAN artifacts
dbscan       = joblib.load( "../models/riskyzones/risky_location_model_dbscan.pkl")
cw_dbscan    = joblib.load("../models/riskyzones/cluster_weights_dbscan.pkl")
cf_dbscan    = joblib.load("../models/riskyzones/cluster_fatalities_dbscan.pkl")
cet_dbscan   = joblib.load("../models/riskyzones/cluster_event_types_dbscan.pkl")

#OPTICS artifacts
optics       = joblib.load("../models/riskyzones/risky_location_model_optics.pkl")
cw_optics    = joblib.load("../models/riskyzones/cluster_weights_optics.pkl")
cf_optics    = joblib.load("../models/riskyzones/cluster_fatalities_optics.pkl")
cet_optics   = joblib.load("../models/riskyzones/cluster_event_types_optics.pkl")
#the coordinate scaler and scaled data
scaler = joblib.load("../models/riskyzones/scaler__risky_loc.pkl")
X_scaled = joblib.load("../models/riskyzones/X_scaled.pkl")


In [ ]:
# Load models for user 1(as a prototype user)
# 2) Dynamic‐threshold model (RandomForestRegressor)
thr_model = joblib.load("../models/user1_threshold_model.pkl")

# 3) Incident‐prediction model + scaler
inc_model   = joblib.load("../models/user1_xgboost.pkl")
inc_scaler  = joblib.load("../models/user1_xgboost_scaler.pkl")

# 4) Audio‐stress model
audio_model = joblib.load("../models/user1_audio_stress_model.pkl")

# 5) Keyword‐spotting model
kw_model    = joblib.load("../models/user1_keyword_spotter.pkl")

# (Optionally) 6) Movement‐anomaly clustering if it’s separate
move_clust, move_scaler = joblib.load("../models/user1_movement_cluster.pkl"), \
                          joblib.load("../models/user1_movement_scaler.pkl")


In [ ]:

# Cell 2: Test Scenario 1 - Normal Case
print("=== Testing Normal Case ===")
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Result:", result)

# Cell 3: Test Scenario 2 - High-Risk Case
print("\n=== Testing High-Risk Case ===")
def detect_emergency_keyword(user_id):  # Override for high-risk
    return 1.0
def predict_audio_stress(user_id):  # Override for high-risk
    return 0.9
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("High-Risk Result:", result)

# Cell 4: Test Scenario 3 - Missing Data
print("\n=== Testing Missing Data ===")
def predict_crime_risk(lat, lon):  # Simulate missing data
    return 0.0
def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.0
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Missing Data Result:", result)

# Cell 5: Test Scenario 4 - Error Case
print("\n=== Testing Error Case ===") 
def capture_and_store(user_id, lat, lon, ts_collection, geo_collection, users_collection):
    raise Exception("Database connection failed")
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Error Case Result:", result)

In [ ]:
# Cell 6: Test Scenario 5 - Low-Risk Case
print("\n=== Testing Low-Risk Case ===")
def predict_crime_risk(lat, lon):
    return 0.1
def detect_movement_anomaly(user_id, ts_collection, lat, lon, now):
    return 0.1
def predict_audio_stress(user_id):
    return 0.1
def detect_emergency_keyword(user_id):
    return 0.0
def detect_user_anomalies(lat, lon, hour, weekday, month, user_id, ts_collection, prob_threshold):
    return 0.0, 0.1
def predict_incident(model, scaler, loc_anomaly, time_anomaly):
    return 0.1
result = process_capture("user123", 40.7128, -74.0060, ts_collection, geo_collection, users_collection)
print("Low-Risk Result:", result)